In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from joblib import dump

# Carregar os dados
file_path = 'ds.csv'
data = pd.read_csv(file_path)

# Pré-processamento dos dados
# Tratar valores ausentes
data['Classificação habilitação anterior'].fillna(data['Classificação habilitação anterior'].mean(), inplace=True)
data['Km Distância ISEC Lisboa - Concelho Morada Indicada pelo estudante'].fillna(data['Km Distância ISEC Lisboa - Concelho Morada Indicada pelo estudante'].median(), inplace=True)

# Converter dados categóricos em numéricos
categorical_columns = data.select_dtypes(include=['object']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    data[column] = data[column].fillna('Desconhecido')
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalizar ou padronizar os dados
scaler = StandardScaler()
columns_to_scale = data.columns[1:-1]
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Remover a coluna de identificação do aluno
data.drop(columns=['Nº'], inplace=True)

# Seleção de características e divisão dos dados em conjuntos de treinamento e teste
X = data.drop(columns=['Abandono?'])
y = data['Abandono?']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Avaliação do modelo
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, rf_model.predict_proba(X_test), multi_class='ovr')

# Exibir métricas de avaliação
print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}, AUC-ROC: {roc_auc:.2f}")

# Salvar o modelo, LabelEncoders e StandardScaler
dump(rf_model, 'modelo_rf.joblib')
dump(label_encoders, 'label_encoders.joblib')
dump(scaler, 'standard_scaler.joblib')


Accuracy: 0.91, Precision: 0.88, Recall: 0.91, F1: 0.89, AUC-ROC: 0.97


c:\Users\CesarCardoso\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['standard_scaler.joblib']